In [8]:
import spacy
from spacy.matcher import Matcher
from spacy import displacy
from spacy.matcher import DependencyMatcher

nlp = spacy.load('en_core_web_md')
matcher = Matcher(nlp.vocab)

def show_ents(doc):
    if doc.ents:
        for ent in doc.ents:
            print(ent.text+' - '+ent.label_+' - ' +
                  str(spacy.explain(ent.label_)))
    else:
        print('No named entities found.')


In [9]:
# Ruler
ruler = nlp.add_pipe("entity_ruler", before="ner")
patterns = [
    {'label': "ORG", "pattern": "Saba Portal"},
    {'label': "ORG", "pattern": "Saba portal"},
    {'label': "ORG", "pattern": "saba portal"},
    {'label': "ORG", "pattern": "Saba"},
    {'label': "ORG", "pattern": "saba"},
]
ruler.add_patterns(patterns)

# Matcher
matcher = Matcher(nlp.vocab)
saba_patterns = [[{"LOWER": "saba"}]]
matcher.add("Saba", saba_patterns)

# Dependency Matcher
dep_matcher = DependencyMatcher(nlp.vocab)
dep_pattern = [
    [
        {
            "RIGHT_ID": "anchor_saba",
            "RIGHT_ATTRS": {"LOWER": "saba"}
        },
        {
            "LEFT_ID": "anchor_saba",
            "REL_OP": "<<",
            "RIGHT_ID": "saba_action",
            "RIGHT_ATTRS": {"POS": "VERB"}
        },
        {
            "LEFT_ID": "saba_action",
            "REL_OP": ">",
            "RIGHT_ID": "negation",
            "RIGHT_ATTRS": {"DEP": "neg"}
        },
    ],
    [
        {
            "RIGHT_ID": "anchor_saba",
            "RIGHT_ATTRS": {"LOWER": "saba"}
        },
        {
            "LEFT_ID": "anchor_saba",
            "REL_OP": "<<",
            "RIGHT_ID": "saba_action",
            "RIGHT_ATTRS": {"POS": "VERB"}
        },
        {
            "LEFT_ID": "saba_action",
            "REL_OP": "<<",
            "RIGHT_ID": "aux_pre_neg",
            "RIGHT_ATTRS": {"POS": "AUX"}
        },
        {
            "LEFT_ID": "aux_pre_neg",
            "REL_OP": ">",
            "RIGHT_ID": "aux_neg",
            "RIGHT_ATTRS": {"DEP": "neg"}
        },
    ],
    [
        {
            "RIGHT_ID": "anchor_saba",
            "RIGHT_ATTRS": {"LOWER": "saba"}
        },
        {
            "LEFT_ID": "anchor_saba",
            "REL_OP": "<<",
            "RIGHT_ID": "aux_unable",
            "RIGHT_ATTRS": {"POS": "AUX"}
        },
        {
            "LEFT_ID": "aux_unable",
            "REL_OP": ">",
            "RIGHT_ID": "unable",
            "RIGHT_ATTRS": {"LOWER": "unable"}
        },
        {
            "LEFT_ID": "unable",
            "REL_OP": ">>",
            "RIGHT_ID": "saba_action",
            "RIGHT_ATTRS": {"POS": "VERB"}
        },
    ],
]
dep_matcher.add("saba_dep", patterns=dep_pattern)


In [15]:
doc = nlp("I am facing issues accessing saba portal.")
# show_ents(doc)
for sent in doc.sents:
    print (displacy.render(sent, style = "dep"))
    

None


In [16]:
dep_matches = dep_matcher(doc)

actions = []
for match in dep_matches:
    temp_matches = match[1]
    for sub_match in temp_matches:
        if ((sub_match not in actions) and (str(doc[sub_match].pos_) == "VERB")):
            actions.append(doc[sub_match].lemma_)
print(actions)

[]


In [11]:
for token in doc:
    print(f"{token.text:{10}} {token.pos_:{10}} ({token.dep_:{10}} {spacy.explain(token.dep_):{25}})  {token.head}")
    

I          PRON       (nsubj      nominal subject          )  facing
am         AUX        (aux        auxiliary                )  facing
facing     VERB       (ROOT       root                     )  facing
issues     NOUN       (dobj       direct object            )  facing
accessing  VERB       (acl        clausal modifier of noun (adjectival clause))  issues
saba       PROPN      (compound   compound                 )  portal
portal     NOUN       (dobj       direct object            )  accessing
.          PUNCT      (punct      punctuation              )  facing
